# Description

* Creating OTUs with usearch

# Init

In [1]:
workDir = '/home/bryan/RhizCG/data/MiSeq_amplicon/150520_run1/OTU_binning/'
databaseDir = '/home/bryan/RhizCG/data/databases/'
nprocs = 24

In [2]:
import os
import re
import pandas as pd
from qiime.assign_taxonomy import UclustConsensusTaxonAssigner
from IPython.display import Image

In [3]:
%pylab inline
%load_ext rpy2.ipython

Populating the interactive namespace from numpy and matplotlib


In [4]:
%%R
library(ggplot2)
library(dplyr)
library(tidyr)
library(gridExtra)
library(stringr)

/opt/anaconda/lib/python2.7/site-packages/rpy2/robjects/functions.py:106: UserWarning: 
Attaching package: ‘dplyr’


  res = super(Function, self).__call__(*new_args, **new_kwargs)
/opt/anaconda/lib/python2.7/site-packages/rpy2/robjects/functions.py:106: UserWarning: The following objects are masked from ‘package:stats’:

    filter, lag


  res = super(Function, self).__call__(*new_args, **new_kwargs)
/opt/anaconda/lib/python2.7/site-packages/rpy2/robjects/functions.py:106: UserWarning: The following objects are masked from ‘package:base’:

    intersect, setdiff, setequal, union


  res = super(Function, self).__call__(*new_args, **new_kwargs)


In [5]:
if not os.path.isdir(workDir):
    os.mkdir(workDir)

# Binning

In [6]:
%%bash -s "$workDir"

cd $1

ln -f -s ../QC/finalQC.fasta .

printf "Number of sequences in final QC fasta: "
grep -c ">" finalQC.fasta

head -n 4 finalQC.fasta
tail -n 4 finalQC.fasta

Number of sequences in final QC fasta: 7912449
>sample3-4.15_Nneg_0
TCCGTAGGGGGCAAGCGTTGTCCGGAATCATTGGGCGTAAAGCGCGCGTAGGCGGCCCGGTAAGTCCGCTCTTAAAGCCCAAGGCTCAACCTTGGGATGGGGTGGATACCGCCGGGCGCGAGTCCGGAAGAGGCGAGTGGAATTCCTGGTGTAGCGGTGGAATGCGCAGATATCAGGAGGAACACCAATAGCGAAGGCAGCTCGCTGGGACGGTACTGACGCTAAGGCGCGAAAGCGTGGGGAGCGAACAGG
>sample5-1.06_Nneg_1
TACGTAGGGGGCAAGCGTTGTCCGGATTTATTGGGCGTAAAGCGCGCGCAGGCGGCCAGGTAGGTCCGCTGTGAAAGCTCCAGGCTCAACCTCGAGACGTCAGCGGAAACCCTCTGGCTAGAGTCCGGAAGAGGAGAGTGGAATTCCTGGTGTAGCGGTGAAATGCGCAGATATCAGGAAGAACACCCGTGGCGAAGGCGGCTCTCTGGGACGGTACTGACGCTGAGGCGCGAAAGCGTGGGGAGCGAACAGG
>sample5-3.13_Nneg_8037684
GACAGAGGATGCAATCGTTATCCGGAATGATTGGGCGTAAAGCGACTGTAGGCGGCTGTTCAAGTCCGCCGTCAAATCCCAGGGCTCAACCCTGGACAGGCGGGGGATACTTCCAAGCTGGAGTACGGTAGGGGCAGAGGGAATTTCCGGTGGAGCGGGGAAATGCGGTGAGATCGGAAAGAACACCAGCGGCGAAGGCAGTCTGCTGGGCCGACACGGACGCTGAGGGGCGAAAGCTAGGGGAGCAAAAGGG
>sample1-2.05_Npos_8037685
TACGTAGGGCGCAAGCGTTATCCGGAATTAGTGGGCGTAAAGAGCTCGTAGGCGGTGTGTCGCGTCTGCTGTGAAATCCCGAGGCGCAACCTCGGG

# Usearch pipeline

In [7]:
!cd $workDir; \
    usearch -derep_fulllength finalQC.fasta -output finalQC_uniques.fasta -sizeout -threads $nprocs

usearch v7.0.1090_i86linux32, 4.0Gb RAM (132Gb total), 40 cores
(C) Copyright 2013 Robert C. Edgar, all rights reserved.
http://drive5.com/usearch

Licensed to: chuck.peperanney@gmail.com

00:12 2.5Gb  100.0% Reading finalQC.fasta

Out of memory mymalloc(31649796), curr 4.27e+09 bytes

myutils.cpp(2212): 
Out of memory mymalloc(9531596), curr 4.27e+09 bytes

myutils.cpp(2212): 
Out of memory mymalloc(9577996), curr 4.27e+09 bytes

myutils.cpp(2212): 
Out of memory mymalloc(9338636), curr 4.27e+09 bytes

myutils.cpp(2212): 
Out of memory mymalloc(12584524), curr 4.27e+09 bytes

myutils.cpp(2212): 


Out of memory mymalloc(9681004), curr 4.27e+09 bytes

myutils.cpp(2212): 
Out of memory mymalloc(23393476), curr 4.27e+09 bytes

myutils.cpp(2212): usearch -derep_fulllength finalQC.fasta -output finalQC_uniques.fasta -sizeout -threads 24
Out of memory mymalloc(10076012), curr 4.27e+09 bytes
Out of memory mymalloc(9838964), curr 4.27e+09 bytes

myutils.cpp(2212): 
Out of memory mymalloc(9364

In [8]:
!cd $workDir; \
    usearch -sortbysize finalQC_uniques.fasta \
    -output finalQC_uniques_sorted.fasta -minsize 2  #-minsize 2

usearch v7.0.1090_i86linux32, 4.0Gb RAM (132Gb total), 40 cores
(C) Copyright 2013 Robert C. Edgar, all rights reserved.
http://drive5.com/usearch

Licensed to: chuck.peperanney@gmail.com



myutils.cpp(195): 

usearch -sortbysize finalQC_uniques.fasta -output finalQC_uniques_sorted.fasta -minsize 2

---Fatal error---
Cannot open finalQC_uniques.fasta, errno=2 No such file or directory


In [9]:
!cd $workDir; \
    tail finalQC_uniques_sorted.fasta

tail: cannot open `finalQC_uniques_sorted.fasta' for reading: No such file or directory


In [10]:
!cd $workDir; \
    usearch -cluster_otus finalQC_uniques_sorted.fasta -otus otus.fasta

usearch v7.0.1090_i86linux32, 4.0Gb RAM (132Gb total), 40 cores
(C) Copyright 2013 Robert C. Edgar, all rights reserved.
http://drive5.com/usearch

Licensed to: chuck.peperanney@gmail.com



myutils.cpp(195): 

usearch -cluster_otus finalQC_uniques_sorted.fasta -otus otus.fasta

---Fatal error---
Cannot open finalQC_uniques_sorted.fasta, errno=2 No such file or directory


In [11]:
!cd $workDir; \
    /opt/bioinfo/edgar_python_scripts/fasta_number.py otus.fasta OTU. > otusn.fasta

Traceback (most recent call last):
  File "/opt/bioinfo/edgar_python_scripts/fasta_number.py", line 28, in <module>
    File = open(sys.argv[1])
IOError: [Errno 2] No such file or directory: 'otus.fasta'


# Removing sequences classified as chloroplast, eukaryotes, or archaea

In [12]:
%%bash -s "$workDir" "$databaseDir" "$nprocs"

cd $1

parallel_assign_taxonomy_uclust.py \
    -r $2'97_Silva_111_rep_set_no_ambig.fasta' \
    -t $2'Silva_111_taxa_map_full.txt' \
    -O 8 \
    -i otusn.fasta \
    -o otusn_tax

Traceback (most recent call last):
  File "/opt/anaconda/bin/parallel_assign_taxonomy_uclust.py", line 91, in <module>
    main()
  File "/opt/anaconda/bin/parallel_assign_taxonomy_uclust.py", line 87, in main
    suppress_submit_jobs=opts.suppress_submit_jobs)
  File "/opt/anaconda/lib/python2.7/site-packages/qiime/parallel/util.py", line 131, in __call__
    working_dir)
  File "/opt/anaconda/lib/python2.7/site-packages/qiime/parallel/util.py", line 482, in _split_fasta
    job_prefix,working_dir=output_dir)
  File "/opt/anaconda/lib/python2.7/site-packages/qiime/split.py", line 100, in split_fasta
    raise ValueError("seqs_per_file must be > 0!")
ValueError: seqs_per_file must be > 0!


In [13]:
%%bash -s "$workDir" "$databaseDir"

cd $1

egrep "Chloroplast|Eukaryota|Archaea|Unassigned|mitochondria" \
    otusn_tax/otusn_tax_assignments.txt |\
    awk '{print $1}' > to_remove_tax.accnos

egrep: otusn_tax/otusn_tax_assignments.txt: No such file or directory


In [14]:
!cd $workDir; wc -l to_remove_tax.accnos

0 to_remove_tax.accnos


In [15]:
!cd $workDir; head to_remove_tax.accnos

In [16]:
!cd $workDir; \
    mothur "#remove.seqs(fasta=otusn.fasta, accnos=to_remove_tax.accnos)" | head -n 50







mothur v.1.35.1
Last updated: 03/31/2015

by
Patrick D. Schloss

Department of Microbiology & Immunology
University of Michigan
pschloss@umich.edu
http://www.mothur.org

When using, please cite:
Schloss, P.D., et al., Introducing mothur: Open-source, platform-independent, community-supported software for describing and comparing microbial communities. Appl Environ Microbiol, 2009. 75(23):7537-41.

Distributed under the GNU General Public License

Type 'help()' for information on the commands that are available

Type 'quit()' to exit program



mothur > remove.seqs(fasta=otusn.fasta, accnos=to_remove_tax.accnos)
[ERROR]: to_remove_tax.accnos is blank, aborting.
You have no valid accnos file and accnos is required.
[ERROR]: otusn.fasta is blank, aborting.
You must provide at least one of the following: fasta, name, group, taxonomy, quality, alignreport, fastq or list.
[ERROR]: did not complete remove.seqs.

mothur > quit()


In [17]:
!cd $workDir; grep -c ">" otusn.fasta
!cd $workDir; grep -c ">" otusn.pick.fasta

0
grep: otusn.pick.fasta: No such file or directory


# Mapping reads

In [18]:
%%bash -s "$workDir"

cd $1

perl -pe 's/^>(.+)(_[^_]+)\n$/>$1$2;barcodelabel=$1\n/' finalQC.fasta > finalQC_usearchfmt.fasta

In [19]:
!cd $workDir; head finalQC_usearchfmt.fasta

>sample3-4.15_Nneg_0;barcodelabel=sample3-4.15_Nneg
TCCGTAGGGGGCAAGCGTTGTCCGGAATCATTGGGCGTAAAGCGCGCGTAGGCGGCCCGGTAAGTCCGCTCTTAAAGCCCAAGGCTCAACCTTGGGATGGGGTGGATACCGCCGGGCGCGAGTCCGGAAGAGGCGAGTGGAATTCCTGGTGTAGCGGTGGAATGCGCAGATATCAGGAGGAACACCAATAGCGAAGGCAGCTCGCTGGGACGGTACTGACGCTAAGGCGCGAAAGCGTGGGGAGCGAACAGG
>sample5-1.06_Nneg_1;barcodelabel=sample5-1.06_Nneg
TACGTAGGGGGCAAGCGTTGTCCGGATTTATTGGGCGTAAAGCGCGCGCAGGCGGCCAGGTAGGTCCGCTGTGAAAGCTCCAGGCTCAACCTCGAGACGTCAGCGGAAACCCTCTGGCTAGAGTCCGGAAGAGGAGAGTGGAATTCCTGGTGTAGCGGTGAAATGCGCAGATATCAGGAAGAACACCCGTGGCGAAGGCGGCTCTCTGGGACGGTACTGACGCTGAGGCGCGAAAGCGTGGGGAGCGAACAGG
>sample6-4.27_Nneg_2;barcodelabel=sample6-4.27_Nneg
TACGTAGGGTGCAAGCGTTAATCGGAATTACTGGGCGTAAAGCGTACGCAGGCGGCTTTGCAAGACAGGTGTGAAATCCCCGGGCCTAACCTGGGAACTGCATTTGTGACTGCATGGCTAGAGTTTGTCAGAGGGGGGTGGAATTCCACGTGTAGCAGTGAAATGCGTAGATATGCGGAGGAACACCGATGGCGAAGGCAATCCCCTGGACCTGTACTGACGCTCATGCACGAAAGCGTGGGGAGCAAACAGG
>sample6-3.07_Nneg_78383;barcodelabel=sample6-3.07_Nneg
TACGTAGGGTGCAAGCGTTAATCGGAA

In [20]:
!cd $workDir; grep -c ">" finalQC_usearchfmt.fasta

7912449


In [21]:
!cd $workDir; \
usearch -usearch_global finalQC_usearchfmt.fasta \
    -db otusn.pick.fasta \
    -strand plus -id 0.97 \
    -uc readmap.uc \
    -threads $nprocs

usearch v7.0.1090_i86linux32, 4.0Gb RAM (132Gb total), 40 cores
(C) Copyright 2013 Robert C. Edgar, all rights reserved.
http://drive5.com/usearch

Licensed to: chuck.peperanney@gmail.com



myutils.cpp(195): 

usearch -usearch_global finalQC_usearchfmt.fasta -db otusn.pick.fasta -strand plus -id 0.97 -uc readmap.uc -threads 24

---Fatal error---
Cannot open otusn.pick.fasta, errno=2 No such file or directory


In [22]:
!cd $workDir; \
    python /opt/bioinfo/edgar_python_scripts/uc2otutab.py readmap.uc > otu_table.txt

readmap.uc 100.0%   


In [23]:
%%bash -s "$workDir"

cd $1

if [ -f otu_table.biom ]; then
    rm otu_table.biom
fi #This is to mitigate a biom bug

biom convert -i otu_table.txt -o otu_table.biom --table-type "otu table"

Traceback (most recent call last):
  File "/opt/anaconda/bin/pyqi", line 177, in <module>
    optparse_main(cmd_obj, argv[1:])
  File "/opt/anaconda/lib/python2.7/site-packages/pyqi/core/interfaces/optparse/__init__.py", line 276, in optparse_main
    result = optparse_cmd(local_argv[1:])
  File "/opt/anaconda/lib/python2.7/site-packages/pyqi/core/interface.py", line 40, in __call__
    cmd_result = self.CmdInstance(**cmd_input)
  File "/opt/anaconda/lib/python2.7/site-packages/pyqi/core/command.py", line 137, in __call__
    result = self.run(**kwargs)
  File "/opt/anaconda/lib/python2.7/site-packages/biom/commands/table_converter.py", line 219, in run
    raise CommandError(convert_error_msg)
pyqi.core.exception.CommandError: Input does not look like a classic table. Did you forget to specify that a classic table file should be created from a BIOM table file?


In [24]:
%%bash -s "$workDir"

cd $1

if [ -f otu_table_summary.txt ]; then
    rm otu_table_summary.txt
fi #This is to mitigate a biom bug

biom summarize-table -i otu_table.biom -o otu_table_summary.txt

Usage: biom summarize-table [options] {-i/--input-fp INPUT-FP -o/--output-fp OUTPUT-FP}

[] indicates optional input (order unimportant)
{} indicates required input (order unimportant)

Provides details on the observation counts per sample, including summary statistics, as well as metadata categories associated with samples and observations.

Example usage: 
Print help message and exit
 biom summarize-table -h

Basic script usage: Write a summary of table.biom to table_summary.txt
 biom summarize-table -i table.biom -o table_summary.txt

biom summarize-table: error: option -i: file does not exist: 'otu_table.biom'


In [25]:
!cd $workDir; cat otu_table_summary.txt

cat: otu_table_summary.txt: No such file or directory


# Assessing the taxonomic composition in the water controls

In [26]:
%%R -i workDir

setwd(workDir)

tbl.otu = read.delim('otu_table.txt', sep='\t')
tbl.tax = read.delim('otusn_tax/otusn_tax_assignments.txt', header=F)

tbl.otu.H2O = tbl.otu %>% 
    select(OTUId, Negative_Control) %>%
    filter(Negative_Control > 0)


Error in file(file, "rt") : cannot open the connection


/opt/anaconda/lib/python2.7/site-packages/rpy2/robjects/functions.py:106: UserWarning: Error in file(file, "rt") : cannot open the connection

  res = super(Function, self).__call__(*new_args, **new_kwargs)
/opt/anaconda/lib/python2.7/site-packages/rpy2/robjects/functions.py:106: UserWarning: In addition: 
  res = super(Function, self).__call__(*new_args, **new_kwargs)
/opt/anaconda/lib/python2.7/site-packages/rpy2/robjects/functions.py:106: UserWarning: Warning message:

  res = super(Function, self).__call__(*new_args, **new_kwargs)
/opt/anaconda/lib/python2.7/site-packages/rpy2/robjects/functions.py:106: UserWarning: In file(file, "rt") :
  res = super(Function, self).__call__(*new_args, **new_kwargs)
/opt/anaconda/lib/python2.7/site-packages/rpy2/robjects/functions.py:106: UserWarning: 
 
  res = super(Function, self).__call__(*new_args, **new_kwargs)
/opt/anaconda/lib/python2.7/site-packages/rpy2/robjects/functions.py:106: UserWarning:  cannot open file 'otusn_tax/otusn_tax_assign

In [27]:
%%R
# taxonomic levels
tbl.j = left_join(tbl.otu.H2O, tbl.tax, by=c('OTUId' = 'V1'))

add_levels = function(x){
    x = as.character(x)
    x = gsub('__', '', x)
    y = str_split(x, '; ')
    y.len = length(y[[1]])
    for(i in y.len:7){
        if(i != 7){
            x = paste(c(x, 'unclassified'), collapse='; ')
            }
        }
    return(x)
    }

tbl.j = tbl.j %>%
    mutate(V2 = sapply(V2, add_levels)) %>%
    separate(V2, c('domain', 'phylum', 'class', 'order', 'family', 'genus', 'species'), sep='; ')


Error in left_join(tbl.otu.H2O, tbl.tax, by = c(OTUId = "V1")) : 
  object 'tbl.otu.H2O' not found


/opt/anaconda/lib/python2.7/site-packages/rpy2/robjects/functions.py:106: UserWarning: Error in left_join(tbl.otu.H2O, tbl.tax, by = c(OTUId = "V1")) : 
  object 'tbl.otu.H2O' not found

  res = super(Function, self).__call__(*new_args, **new_kwargs)


In [28]:
%%R -h 600 -w 600

tbl.j = tbl.j %>%
    arrange(Negative_Control)

tbl.j.phylum = tbl.j %>%
    group_by(domain, phylum) %>%
    summarize(
        OTU_count = n(),
        abundance = sum(Negative_Control)
    )

p1 = ggplot(tbl.j.phylum, aes(phylum, OTU_count, color=domain)) +
    geom_point() +
    theme(
        axis.text.x = element_text(angle=90, hjust=1)
        )


tbl.j.class = tbl.j %>%
    group_by(class) %>%
    summarize(
        OTU_count = n(),
        abundance = sum(Negative_Control)
    )

p2 = ggplot(tbl.j.class, aes(class, OTU_count, color=class)) +
    geom_point() +
    theme(
        axis.text.x = element_text(angle=90, hjust=1)
        )


tbl.j.genus = tbl.j %>%
    group_by(class, genus) %>%
    summarize(
        OTU_count = n(),
        abundance = sum(Negative_Control)
    )

p3 = ggplot(tbl.j.genus, aes(genus, OTU_count, color=class)) +
    geom_point() +
    theme(
        axis.text.x = element_text(angle=90, hjust=1)
        )

grid.arrange(p1, p2, p3, ncol=1)


Error in eval(expr, envir, enclos) : object 'tbl.j' not found


/opt/anaconda/lib/python2.7/site-packages/rpy2/robjects/functions.py:106: UserWarning: Error in eval(expr, envir, enclos) : object 'tbl.j' not found

  res = super(Function, self).__call__(*new_args, **new_kwargs)
